# NiceCX v1.0 Tutorial

In this tutorial you will learn to use NiceCX, a simple data model that is part of the ndex2 NDEx Client module.
NiceCX facilitates creating and working with networks, including interfaces to NetworkX and Pandas.
This tutorial requires Python 3.6+ and the ndex2 module, see the NDEx Client Tutorial for installation instructions.

### Import Packages Required for this Tutorial

In [8]:
from nicecxModel.NiceCXNetwork import NiceCXNetwork
from nicecxModel.cx.aspects.NodesElement import NodesElement
from nicecxModel.cx.aspects.EdgesElement import EdgesElement
from nicecxModel.cx.aspects.NodeAttributesElement import NodeAttributesElement
from nicecxModel.cx.aspects.EdgeAttributesElement import EdgeAttributesElement
import ndex2.client as nc
import networkx as nx
import pandas as pd

# Five Ways to Work with NiceCX networks:

- Starting with an Empty Network
- CX Files
- NDEx Networks
- NetworkX Networks
- Pandas DataFrames

## Starting with an Empty Network

**Create an empty niceCx network**

In [2]:
niceCx_creatures = NiceCXNetwork()

**Name the Network**

The _niceCx_creatures_ will now be populated with data in which each node represents a species and has a color attribute. Each edge will specify a relationship between the two species. First, we will set the name of the network:  

In [3]:
niceCx_creatures.setName("Food Web")

**Add Nodes and Edges**

In [4]:
fox_node = niceCx_creatures.addNode(node_name='Fox')
mouse_node = niceCx_creatures.addNode(node_name='Mouse')
bird_node = niceCx_creatures.addNode(node_name='Bird')

fox_bird_edge = niceCx_creatures.addEdge(edge_source=fox_node, edge_target=bird_node, edge_interaction='interacts-with')

fox_mouse_edge = niceCx_creatures.addEdge(edge_source=fox_node, edge_target=mouse_node, edge_interaction='interacts-with')


The addNode and addEdge methods return the unique ID assigned to the new node or edge. In CX, IDs are always assigned in an ascending order, although they may not always be sequential In this case, the node with the name "Fox" will have an ID of 0, "Mouse" will have 1, and so on. The edge between "Fox" and "Bird" will have the ID of 0, the next will be 1, and so on. 

**Add Attributes**

The addNodeAttribute and addEdgeAttribute require the ID in the "property_of" field and the property "name" and "values" in their respective fields.

In [5]:
niceCx_creatures.addNodeAttribute(property_of=fox_node, name='Color', values='Red')

niceCx_creatures.addNodeAttribute(property_of=mouse_node, name='Color', values='Gray')

niceCx_creatures.addNodeAttribute(property_of=bird_node, name='Color', values='Blue')

In [6]:
niceCx_creatures.addEdgeAttribute(property_of=fox_mouse_edge, name='Hunted', values='On the ground')

We can now print a summary of niceCX_creatures. The returned value of the getSummary() method is structured in the same format as NetworkSummary objects returned by NDEx network search methods.

In [7]:
print(niceCx_creatures.getSummary())

Name: Food Web
Nodes: 3
Edges: 2
Node Attributes: 3
Edge Attributes: 1



## CX Files

In [8]:
niceCx_from_cx_file = NiceCXNetwork(filename='SimpleNetwork.cx')
print(niceCx_from_cx_file.getSummary())

Name: 
Nodes: 2
Edges: 1
Node Attributes: 0
Edge Attributes: 0



## NDEx Networks

In [3]:
niceCx_from_server = NiceCXNetwork(server='public.ndexbio.org', uuid='f1dd6cc3-0007-11e6-b550-06603eb7f303')
print(niceCx_from_server.getSummary())

http://public.ndexbio.org/v2/network/f1dd6cc3-0007-11e6-b550-06603eb7f303/aspect
Name: CoCaNet2
Nodes: 36
Edges: 37
Node Attributes: 778
Edge Attributes: 659



## NetworkX Networks

Networks can be manipulated using NetworkX facilities and then used to create a NiceCx network. Here we create a NetworkX network from scratch:

In [13]:
G = nx.Graph()
G.add_node('ABC')
G.add_node('DEF')
G.add_node('GHI')
G.add_node('JKL')
G.add_node('MNO')
G.add_node('PQR')
G.add_node('XYZ')
G.add_edges_from([('ABC','DEF'), ('DEF', 'GHI'),('GHI', 'JKL'), 
                  ('DEF', 'JKL'), ('JKL', 'MNO'), ('DEF', 'MNO'),
                 ('MNO', 'XYZ'), ('DEF', 'PQR')])

We now use NetworkX to find the shortest path between nodes ABC and MNO and then create a subgraph from that path.

In [14]:
short_path = nx.shortest_path(G,source='ABC',target="MNO")

path_subgraph = G.subgraph(short_path)

Finally, NiceCx objects are created from both *G* and *path_subgraph*.

In [15]:
niceCx_full = NiceCXNetwork(networkx_G=G)
niceCx_full.setName('Created from NetworkX (full)')

niceCx_short = NiceCXNetwork(networkx_G=path_subgraph)
niceCx_short.setName('Created from NetworkX (shortest path)')
                       
print(niceCx_full.getSummary())
print(G.edges())
print('')

print(niceCx_short.getSummary())
print(path_subgraph.edges())

Name: Created from NetworkX (full)
Nodes: 7
Edges: 8
Node Attributes: 0
Edge Attributes: 0

[('ABC', 'DEF'), ('DEF', 'GHI'), ('DEF', 'JKL'), ('DEF', 'MNO'), ('DEF', 'PQR'), ('GHI', 'JKL'), ('JKL', 'MNO'), ('MNO', 'XYZ')]

Name: Created from NetworkX (shortest path)
Nodes: 3
Edges: 2
Node Attributes: 0
Edge Attributes: 0

[('ABC', 'DEF'), ('DEF', 'MNO')]


NetworkX networks can be created from NiceCX. Only the 

## Pandas DataFrames

### 2 Column DataFrame with No Headers:

In [16]:
data = [('ABC', 'DEF'), ('DEF', 'XYZ')]

df = pd.DataFrame.from_records(data)

niceCx_df_2_column = NiceCXNetwork(pandas_df=df)

print(niceCx_df_2_column.getSummary())

df_2_col_from_niceCx = niceCx_df_2_column.to_pandas()
print(df_2_col_from_niceCx)

Name: Pandas Upload
Nodes: 3
Edges: 2
Node Attributes: 0
Edge Attributes: 0

  source     interaction target
0    ABC  interacts-with    DEF
1    DEF  interacts-with    XYZ


### 3 Column DataFrame with No Headers:

In [17]:
data = [('ABC', 'DEF', 'interacts-with'), ('DEF', 'XYZ', 'neighbor-of')]

df = pd.DataFrame.from_records(data)

niceCx_df_3_column = NiceCXNetwork(pandas_df=df)

print(niceCx_df_3_column.getSummary())

df_3_col_from_niceCx = niceCx_df_3_column.to_pandas()
print(df_3_col_from_niceCx)

Name: Pandas Upload
Nodes: 3
Edges: 2
Node Attributes: 0
Edge Attributes: 0

  source     interaction target
0    ABC  interacts-with    DEF
1    DEF     neighbor-of    XYZ


### 3+ Column DataFrame with Headers to Specify Attribute Columns

In [18]:
df = pd.DataFrame.from_items([('Source', ['ABC', 'DEF']),
                              ('Target', ['DEF', 'XYZ']),
                              ('Interaction', ['interacts-with', 'neighbor-of']),
                              ('EdgeProp', ['Edge property 1', 'Edge property 2'])])

niceCx_df_with_headers = NiceCXNetwork()

niceCx_df_with_headers.create_from_pandas(df, source_field='Source', target_field='Target', 
                          edge_attr=['EdgeProp'], edge_interaction='Interaction')

print(niceCx_df_with_headers.getSummary())

df_headers_from_niceCx = niceCx_df_with_headers.to_pandas()
print(df_headers_from_niceCx)

0
Name: Pandas Upload
Nodes: 3
Edges: 2
Node Attributes: 0
Edge Attributes: 2

  source     interaction target         EdgeProp
0    ABC  interacts-with    DEF  Edge property 1
1    DEF     neighbor-of    XYZ  Edge property 2


### Exporting a Larger and More Complex Network

More complex networks can be output to a Pandas DataFrame as well. In the following example we convert the example network we loaded from the server.

In [19]:
niceCx_from_server = niceCx_from_server.to_pandas()
print(niceCx_from_server)

            source interaction          target IsBidirectional   yeast1  \
0      DHFR (DFR1)          pp    MAPK1 (FUS3)                  YOR236W   
1     CHEK1 (CHK1)          pp      BLM (SGS1)                  YBR274W   
2    IMPDH1 (IMD4)          pp    MTIF2 (IFM1)                  YML056C   
3    IMPDH1 (IMD4)          pp    ING5 (PHO23)                  YML056C   
4    IMPDH1 (IMD4)          pp    ING4 (PHO23)                  YML056C   
5     MAPK1 (FUS3)          pp      NF1 (IRA1)                  YBL016W   
6     HDAC2 (RPD3)          pp     WEE1 (SWE1)             yes  YNL330C   
7     HDAC1 (RPD3)          pp     WEE1 (SWE1)             yes  YNL330C   
8   CSNK1G1 (YCK2)          pp   SH3GL1 (NBP2)                  YNL154C   
9   CSNK1G1 (YCK2)          pp     FZR1 (CDH1)                  YNL154C   
10   MAP2K1 (PBS2)          pp    MAPK1 (FUS3)                  YJL128C   
11   MAP2K1 (PBS2)          pp   RAD52 (RAD52)                  YJL128C   
12    HDAC6 (HDA1)       

Likewise, networks can be output as a networkx graph.

In [5]:
niceCx_from_server_to_netx = niceCx_from_server.to_networkx()
print(niceCx_from_server_to_netx)

CoCaNet2


**To continue with the tutorial, you must edit the following cell to replace the values of the ‘my_account’ and ‘my_password’ variables with a real NDEx account name and password.**

In [9]:
my_account="my_account"
my_password="my_password"
my_server="http://public.ndexbio.org"
try:
    my_ndex=nc.Ndex2(my_server, my_account, my_password)
    my_ndex.update_status()
    print("Success.  Please continue.")
except Exception as inst:
    print("Could not access account %s with password %s" % (my_account, my_password))
    print(inst.args)

Could not access account my_account with password my_password
(u'401 Client Error:  for url: http://public.ndexbio.org/v2/admin/status',)


NiceCx networks can be saved to the NDEx server by calling **upload_to()**

In [ ]:
upload_message = niceCx_df_with_headers.upload_to(my_server, my_account, my_password)
print(upload_message)